In [ ]:
#  Pipeline pour l’encodage FHIR de données gériatriques textuelles

In [ ]:
# LICENCE

# 0. Comité d'éthique pour IA dans QualiFHIR : # Comité d'éthique : G2-2024-E012
# 1. a. tests sur le nouveau serveur de test de Frédérik pour envoyer des ressources FHIR
#    b. serveur de PROD = IRIS 

# 2. développement Python pour travailler avec le modèle de MISTRAL AI : extraction des concepts dans le textre libre 
# 3. validation des codes snomed ct des concepts extraits avec le serveur du Ministère 

# Code Python - Copyright (c) 2025-2026 Matisse Bornard & Dr. Marie DETRAIT at Grand Hôpital de Charleroi

# This script is not intended for sale. 
# Additional Requirements:
# You may not modify the work.
# Any derivative work of this script must include a citation of the original author in any version distributed or 
# publicly displayed of the derivative work.
# Any new work based on this script must be discussed with the author.


# Objectif
L’objectif est de structurer des données gériatriques et ensuite d'autres spécialités  en identifiant les concepts médicaux pertinents pour obtneir une structuration de l'histoire de la maladie
Objectif secondaire : insérer dans des ressources FHIR afin de les stocker dans un entrepôt de données et pouvoir ensuite les exploiter
## Etapes : 
1. Développer un pipeline d’extraction de texte :
   - Identifier les sources de texte (comptes rendus médicaux, observations, etc.).
   - les champs des textes qui sont pertinents pour la tâche
   - Mettre en place un système d’extraction avec un LLM = Le Chat de Mistral AI 
2. Déployer un modèle de traitement du langage naturel (LLM) :
    - Sélectionner et affiner un modèle de NLP adapté aux textes médicaux 
    - Extraire les concepts médicaux (diagnostics, symptômes, traitements) en s’appuyant sur des ontologies médicales (SNOMED CT, LOINC, ATC, etc.).
    - utiliser le serveur terminologique de l'ANS pour réaliser la recherche et les liens
3. Valider les codes avec un serveur terminologique différent du premier et Intégrer les données dans FHIR  
    - valider les codes avec le serveur terminologique du Ministère ( = Bart De Cuypere et David Op De Beeck)
    - Structurer les informations extraites sous forme de ressources FHIR pertinentes (Observation, Condition, etc.).
    - Assurer l’ingestion des données dans notre entrepôt FHIR.
4. Validation et documentation :
    - Tester la robustesse du pipeline sur des données réelles.
    - Documenter le travail réalisé et proposer des améliorations.


In [75]:
#pip install --upgrade pip

In [29]:
!pip --quiet install langchain
!pip --quiet install langchain-mistralai
!pip --quiet install pandas openpyxl
!pip --quiet install python-Levenshtein
!pip --quiet install fuzzywuzzy[speedup]


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from pprint import pprint
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from typing import List
from pydantic import BaseModel, Field
from typing import List, Optional, Literal, Union
from datetime import date
from langchain_core.output_parsers import PydanticOutputParser
from typing import Literal
import pandas as pd
import requests
import time
from datetime import datetime, timezone
from typing import List, Dict
from typing import Callable
from typing import List, Optional
import re
from Levenshtein import ratio as lev_ratio
from fuzzywuzzy import fuzz
import requests
import urllib.parse
from Levenshtein import ratio  # pip install python-Levenshtein
from typing import Dict, Optional, List
import os
import json



### Initialisation du modèle LLM (Mistral)

Nous initialisons une instance du modèle de langage `ChatMistralAI`. Ce modèle sera utilisé pour extraire automatiquement des concepts cliniques à partir de textes médicaux.

Les paramètres utilisés sont les suivants :

- **`model_name`** : nom du modèle déployé (ex. : `mistral-7b` ou une version fine-tunée spécifique au domaine médical).
- **`base_url`** : URL de l'API où le modèle est hébergé.
- **`api_key`** : clé d’authentification sécurisée pour accéder à l’API.
- **`temperature` = 0.3** : contrôle la créativité du modèle (valeur basse = réponses plus déterministes).
- **`max_tokens` = 1000** : limite le nombre maximal de tokens générés par le modèle en sortie.
- **`top_p` = 0.9** : stratégie de filtrage par noyau pour améliorer la pertinence des réponses.

Ce modèle sera ensuite interrogé pour analyser du texte médical et en extraire des éléments exploitables dans des ressources FHIR.

In [79]:
# pour mémoire code de Matisse 
# ----- api_key à générer sur le site de mistral dans la section api -----

#api_key = "" 

#Identifiants pour le SMT ANS (Service de terminologie SNOMED CT)
#    - Nécessite un compte ANS et acceptation de la licence SNOMED CT.
#api_url = "https://api.mistral.ai/v1"
#model_name = "mistral-large-latest"
##model_name = "mistral-large-2402"

#email = ""
#password = ""
#------- URL du serveur FHIR------
#FHIR_SERVER = "http://localhost:8080/fhir"

In [2]:
# ne pas lancer sauf pour les tests
# notre serveur FHIR IRIS (PROD)
# notre serveur de test = HAPI 

import requests
from requests.auth import HTTPBasicAuth

# Coordonnées du serveur FHIR de test (Frédérik)
FHIR_BASE = "http://10.30.3.6:5080/fhir/"
HEADERS = {"Content-Type": "application/fhir+json"}

# Requête pour obtenir tous les patients actuellement sur le serveur de test 
response = requests.get(f"{FHIR_BASE}/Patient?_count=100")  # 

# Vérification du statut et print
if response.status_code == 200:
    bundle = response.json()
    for entry in bundle.get("entry", []):
        patient = entry["resource"]
        print(f"ID: {patient['id']}, Nom: {patient.get('name', [{}])[0].get('family', 'inconnu')}")
        print(f"ID: {patient['id']}, Prénom: {patient.get('name', [{}])[0].get('given', 'inconnu')}")
else:
    print(f"Erreur {response.status_code}: {response.text}")


ID: 1, Nom: QualiFHIR_test
ID: 1, Prénom: ['John']
ID: 2, Nom: QualiFHIR_test
ID: 2, Prénom: ['John']
ID: 37, Nom: CQL oscar
ID: 37, Prénom: inconnu
ID: 41, Nom: CQL madeleine
ID: 41, Prénom: inconnu
ID: 44, Nom: CQL phil
ID: 44, Prénom: inconnu
ID: 49, Nom: QualiFHIR_test_CQL
ID: 49, Prénom: ['Anne']
ID: 67, Nom: Decharleroi
ID: 67, Prénom: ['Tata']
ID: 69, Nom: QUALIFHIR_TEST_CQL
ID: 69, Prénom: ['MADELEINE']
ID: 74, Nom: QUALIFHIR_TEST_CQL
ID: 74, Prénom: ['OSCAR']
ID: 79, Nom: QUALIFHIR_TEST_CQL
ID: 79, Prénom: ['PHIL']
ID: 83, Nom: QUALIFHIR_TEST
ID: 83, Prénom: ['SARAH']
ID: 88, Nom: QUALIFHIR_TEST
ID: 88, Prénom: ['JOHN']
ID: 335, Nom: Qualifhir_Test
ID: 335, Prénom: ['Ludovic']


In [4]:
# CLE Mistral 
api_key = "LLUP0NZsLxMTOWfzGXOiB9lfbb9J8Ktk" 

api_url = "https://api.mistral.ai/v1"
model_name = "mistral-large-latest"

#Identifiants pour le SMT ANS (Service de terminologie SNOMED CT)
#    - Nécessite un compte ANS et acceptation de la licence SNOMED CT.
email = "marie.detrait@ghdc.be"
password = "password"


In [ ]:
# Ministère 

# Minsitère Belge 
https://github.com/ehealthplatformstandards/snowstorm-local

# Snowstorm local (beHealth)


#Infos de Frédérik : 
#Le serveur termino à été installé sur le serveur de prod. il est dans notre envirronnement. 

#Snowstorm local (beHealth)
#    http://10.30.3.205:8180    (fhir)
#    http://10.30.3.205:8181    (browser)

#L'objectif est de venir valider avec le serveur terminologique (avec $lookup) ce que le modèle a trouvé en amont avec ce serveur. 

In [5]:
import requests

url = "http://10.30.3.205:8180/fhir/CodeSystem/$lookup"
params = {    "system": "http://snomed.info/sct",    "code": "247472004", } # Code SNOMED CT pour "urticaire"
resp = requests.get(url, params=params)
#print(resp.status_code, resp.json())

In [6]:
# juste pour voir si cela marche, je valide un code avec $lookup : 
def validate_snomed_code(code: str):
    url = "http://10.30.3.205:8180/fhir/CodeSystem/$lookup"
    params = {"system": "http://snomed.info/sct", "code": code}
    resp = requests.get(url, params=params)
    if resp.status_code == 200:
        return resp.json()  # Détails du code
    else:
        return None  # Code invalide

In [7]:
# avec toutes la hierarchie 
#validate_snomed_code("52967002") " myelofibrose 

In [8]:
model = ChatMistralAI(
    model=model_name, 
    base_url=api_url,
    api_key=api_key,
    temperature = 0.3,
    max_tokens = 1000,
    top_p = 0.9,
)

#### **Authentification au SMT**

en amont se créer un compte 

La fonction get\_snomed\_token permet d’obtenir un jeton d’accès (access\_token) en utilisant le protocole OAuth2 avec le mode d’authentification password. Ce jeton est indispensable pour interagir avec les APIs sécurisées du SMT. L’utilisateur doit fournir son email, son mot de passe, et éventuellement un client\_id. L’URL de connexion est celle du service d’authentification de l’ANS.

Une fois l’accès obtenu, le token est affiché pour vérification.

In [9]:
def get_snomed_token(
    username: str,
    password: str,
    client_id: str = "user-api",
    token_url: str = "https://smt.esante.gouv.fr/ans/sso/auth/realms/ANS/protocol/openid-connect/token"
) -> str:
    """
    Récupère un access_token OAuth2 via le grant_type password pour le SMT ANS.
    
    :param username: ton adresse e-mail (ex. "matisse.bornard@gmail.com")
    :param password: ton mot de passe
    :param client_id: l'ID client (par défaut "user-api")
    :param token_url: URL du point de token (par défaut celui du SMT ANS)
    :return: la chaîne access_token
    :raises HTTPError: si la requête échoue (statut != 2xx)
    """
    headers = {
        "Accept": "*/*",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "username": username,
        "password": password,
        "client_id": client_id,
        "grant_type": "password",
        "refresh_token": ""
    }
    resp = requests.post(token_url, headers=headers, data=data, timeout=5)
    resp.raise_for_status()
    payload = resp.json()
    return payload["access_token"]

token = get_snomed_token(email, password)
print("Access token :", token)

Access token : eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ2ZGZJdTlIZk1Ydm9ZcTFfX0UzWVI4dkFFYnI0OERXbjk5Q2R4TTNUS1BzIn0.eyJleHAiOjE3NTk3NjIwNzcsImlhdCI6MTc1OTc1NjA3NywianRpIjoiMTRhYzUxZmEtYjZjNC00NWY2LTlkN2YtM2U0MTJjMTM5ZDcxIiwiaXNzIjoiaHR0cHM6Ly9zbXQuZXNhbnRlLmdvdXYuZnIvYW5zL3Nzby9hdXRoL3JlYWxtcy9BTlMiLCJhdWQiOlsib2F1dGgyLXJlc291cmNlIiwib250b3NlcnZlciIsInJlYWxtLW1hbmFnZW1lbnQiLCJhY2NvdW50Il0sInN1YiI6IjEzNjE1YTQwLWRmZGYtNDY5OC1iNDZlLTE1NjVjNjc2NTg2NyIsInR5cCI6IkJlYXJlciIsImF6cCI6InVzZXItYXBpIiwic2Vzc2lvbl9zdGF0ZSI6IjhjZWRiOGQ1LTU2ZjAtNGQwZC1hY2FhLTQxNGZhNTFkOWYzOSIsImFsbG93ZWQtb3JpZ2lucyI6WyIvKiJdLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZGVmYXVsdC1yb2xlcy1hbnMiLCJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsib250b3NlcnZlciI6eyJyb2xlcyI6WyJncm91cGluZy9wcm90ZWN0ZWQucmVhZCIsInN5c3RlbS8qLnJlYWQiXX0sInJlYWxtLW1hbmFnZW1lbnQiOnsicm9sZXMiOlsibWFuYWdlLXVzZXJzIl19LCJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJkZWxldGUtYWNjb3Vud

### Fonction de validation des correspondances SNOMED CT

Cette fonction `verify_coding` permet de valider automatiquement une correspondance entre un terme clinique libre et une liste de candidats SNOMED CT, selon plusieurs critères :

1. **Détection de négation**  
   Les termes contenant des préfixes négatifs explicites (`"pas de"`, `"aucun"`, `"sans"`, etc.) sont automatiquement exclus du codage.

2. **Normalisation des textes**  
   Suppression de la casse, ponctuation et espaces pour améliorer la robustesse des comparaisons.

3. **Scoring hybride sur 3 critères** :
   - **Fuzzy matching** (`token_set_ratio` via *fuzzywuzzy*) : on découpe chaque phrase en tokens (mots), on forme deux ensembles,
on mesure la similarité de ces ensembles via un algorithme type Jaccard + pondération.
   - **Levenshtein ratio** (distance de similarité caractère par caractère) : La distance de Levenshtein est le nombre minimal d’insertion/suppression/substitution de caractères
pour passer d’une chaîne à l’autre.

4. **Seuils adaptatifs**  
   Les seuils de validation varient selon la longueur du terme à comparer : plus un terme est long, plus la tolérance augmente.

5. **Sélection finale**  
   Le meilleur candidat validé (au moins 2 critères sur 3) est retourné au format `Coding` FHIR. Sinon, un message de rejet est affiché.

Cette fonction est utilisée pour filtrer et fiabiliser l’intégration des codes SNOMED dans les ressources FHIR.

In [ ]:
# Modification du Prompt pour obtenir un top 3 et le premier du top 3 en sélection ensuite afin de ne rien laisser vide 
# j'ai testé avant évidemment et c'est le premier de la liste qui est correct

In [10]:

# 0) Avant tout : un dictionnaire d’abréviations courantes dans nos textes 
ABBREV_MAP = {
    "bpco": "bronchopneumopathie chronique obstructive",
    "hta":  "hypertension arterielle",
    "HTA": "hypertension arterielle", 
    "PTG" : "Prothèse totale de genou",
    "PTH" : "Prothèse totale de hanche",
    "BAV" : "Bloc auriculo ventriculaire", 
    "FA" : "Fibrilation auriculaire", 
    "TV" : "Tachichardie ventriculaire",
    "FV" : "Fibrilation ventriculaire",
    "anev" : "anévrisme",
    "aeg" : "altération état général",
    "TOT" : "Trans-Obturator Tape",
    "bpco": "bronchopneumopathie chronique obstructive",
    "hta":  "hypertension arterielle",
    "PTG" : "Prothèse totale de genou",
    "PTH" : "Prothèse totale de hanche",
    "BAV" : "Bloc auriculo ventriculaire", 
    "FA" : "Fibrilation auriculaire", 
    "TV" : "Tachycardie ventriculaire",
    "FV" : "Fibrilation ventriculaire",
    "anev" : "anévrisme",
    "aeg" : "altération état général",
    "TOT" : "Trans-Obturator Tape",
    "bpco": "bronchopneumopathie chronique obstructive",
    "hta":  "hypertension arterielle",
    "PTG" : "Prothèse totale de genou",
    "PTH" : "Prothèse totale de hanche",
    "BAV" : "Bloc auriculo ventriculaire", 
    "FA" : "Fibrilation auriculaire", 
    "TV" : "Tachichardie ventriculaire",
    "FV" : "Fibrilation ventriculaire",
    "anev" : "anévrisme",
    "aeg" : "altération état général",
    "TOT" : "Trans-Obturator Tape",
    "LNH" : "lymphome non hodgkinien",
    "FRCV" : "facteurs de risque cardio-vasculaire",
    "HBV" : " virus de l'hépatite B",
    "HCV" : "virus de l'hépatite C",
    "HIV" : "virus de l'immunodéficience humaine",
    "VZV" : "virus de la varicelle/zona",
    "HSV" : "virus de l'herpes simplex",
    "MI" : "membres inférieurs",
    "SUCU" : "sédiment urinaire culture urinaire",
    "ADO" : "antidiabétiques oraux",
    "EEG" : "électroencéphalogramme",
    "EMG" : "électromyographie",
    "Copro" : "coproculture",
    "MRS" : "maison de repos et de soins",
    "MTX" : "méthotrexate",
    "G-CSF" : "facteur de croissance de la lignée granulocytaire",
    "PNP" : "polyneuropathie",
    "ATB" : "antibiothérapie",
    "HdJ" : "hôpital de jour",
    "RCUH" : "recto-colite ulcéro hémorragique", 
    "ADP" : "adénopathie",
    "CAP" : "course à pieds",
    "PCL" :" palpation de la colonne lombaire", 
    "EL": "ebranlement lombaire", 
    "MT": "medecin traitant", 
    "mild cog impair" : "trouble cognitif modéré", 
    "gold" : "classification de Gold", 
    "artic" : "articulations",
    "facies" : " visage", 
    "AG": "anesthésie générale", 

}
# 1) Préfixes de négation à détecter
NEG_PREFIXES = ["pas de ", "aucun ", "sans ", "absence de ", "pas "]


"""def expand_abbrev(term: str) -> str:
    
    #Si term (après normalisation) correspond à une abréviation,
    #renvoie sa forme longue. Sinon renvoie term inchangé.
    
    key = normalize(term)
    return ABBREV_MAP.get(key, term)"""
    
def expand_abbrev(term: str) -> str:
    """
    Remplace **dans la phrase** chaque abréviation connue 
    par sa forme longue.
    """
    # On conserve la ponctuation pour reconstruire plus proprement
    tokens = term.split()
    out_tokens = []
    for tok in tokens:
        key = normalize(tok)
        if key in ABBREV_MAP:
            out_tokens.append(ABBREV_MAP[key])
        else:
            out_tokens.append(tok)
    return " ".join(out_tokens)
    
def normalize(text: str) -> str:
    """Minuscule, supprime ponctuation et espaces superflus."""
    return re.sub(r"[^a-z0-9 ]", "", text.lower()).strip()

def is_negated(term: str) -> bool:
    """Retourne True si le terme commence par une négation connue."""
    tn = term.lower().strip()
    return any(tn.startswith(pref) for pref in NEG_PREFIXES)

def verify_coding(
    term: str,
    candidates: List[Dict],
    lev_thresh: float = 0.8,
    token_thresh: int = 80
) -> Dict | None:
    """
    Sélectionne le meilleur Coding SNOMED CT parmi les candidats.
    En cas d'échec strict, retourne le premier du top 3 des candidats.
    """
    # --- 0) Expand abbr. ---
    expanded = expand_abbrev(term)
    if expanded != term:
        term = expanded

    # --- 1) Négation ? ---
    if is_negated(term):
        print(f"[SNOMED-VERIFY] ⚠️ Négation détectée pour '{term}', pas de codage.")
        return None

    term_n = normalize(term)
    n_words = len(term_n.split())

    # --- 2) Ajustement dynamique des seuils ---
    if n_words >= 6:
        lev_thresh_adj, token_thresh_adj = 0.6, 50
    elif n_words >= 3:
        lev_thresh_adj, token_thresh_adj = 0.7, 60
    else:
        lev_thresh_adj, token_thresh_adj = lev_thresh, token_thresh

    best = {"score": 0.0, "coding": None}

    # --- 3) Parcours des candidats ---
    for c in candidates:
        label = c.get("prefLabel") or c.get("label", "")
        label_n = normalize(label)

        # a) MOT-ENTIER strict
        if not re.search(rf"\b{re.escape(label_n)}\b", term_n):
            continue

        # b) Scores
        token_score = fuzz.token_set_ratio(term_n, label_n)
        lev_score = lev_ratio(term_n, label_n)

        # c) Exige substring + fuzz + lev
        sub_ok = (label_n in term_n) or (term_n in label_n)
        if not (sub_ok and token_score >= token_thresh_adj and lev_score >= lev_thresh_adj):
            continue

        # Garde le meilleur
        if lev_score > best["score"]:
            code = c.get("code") or c.get("id", "").rsplit("/", 1)[-1]
            best["coding"] = {
                "system": "http://snomed.info/sct",
                "code": code,
                "display": label
            }
            best["score"] = lev_score

    # --- 4) Si aucun match strict, retourne le premier du top 3 ---
    if best["coding"] is None and candidates:
        top3 = candidates[:3]  # Prend les 3 premiers candidats
        print(f"[SNOMED-VERIFY] ATTENTION : Aucun match strict pour '{term}'. Sélection du premier du top 3 : {top3[0].get('prefLabel') or top3[0].get('label')}")

        # Retourne le premier candidat du top 3
        first_candidate = top3[0]
        code = first_candidate.get("code") or first_candidate.get("id", "").rsplit("/", 1)[-1]
        label = first_candidate.get("prefLabel") or first_candidate.get("label", "")

        return {
            "system": "http://snomed.info/sct",
            "code": code,
            "display": label
        }

    return best["coding"]


### Fonction d’interrogation du serveur SMT de l’ANS (SNOMED CT)

La fonction `get_snomed_coding` permet d’interroger automatiquement le serveur multi-terminologies (SMT) de l’ANS pour retrouver un concept SNOMED CT correspondant à un terme clinique donné.

Fonctionnement :

1. **Requête API**  
   - Envoie une requête POST à l’API de recherche de concepts avec le terme fourni.
   - Limite la recherche à 10 résultats, en français, sur la terminologie SNOMED CT française.

2. **Filtrage des résultats**  
   - Récupère uniquement les concepts issus de la terminologie `terminologie-snomed-ct-fr`.

3. **Validation de la correspondance**  
   - Utilise la fonction `verify_coding` pour filtrer les concepts selon la similarité avec le terme d’origine (Levenshtein, fuzzy).
   - Ne retourne un objet `Coding` FHIR que si la correspondance est jugée suffisante.

Cette fonction permet d’enrichir les ressources FHIR avec des codes SNOMED de manière semi-automatique, fiable et conforme aux règles d’interopérabilité.

In [84]:

def get_snomed_coding(term: str, token: str, threshold: float = 0.7) -> dict | None:
    """
    Recherche le terme sur SMT ANS, récupère plusieurs concepts SNOMED CT,
    et ne renvoie un Coding FHIR que si la similarité libellé↔term ≥ threshold.
    """
    # 1. Prépare la requête
    base_url = "https://smt.esante.gouv.fr/api/concepts/search"
    params = {
        "searchedText": term,
        "page": 1,
        "size": 10,           # on prend plusieurs candidats
        "lang": "fr",
        "exact": "false"
    }
    query = "&".join(f"{k}={urllib.parse.quote(str(v))}" for k, v in params.items())
    url = f"{base_url}?{query}"

    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    body = [{"terminoName": "terminologie-snomed-ct-fr"}]

    try:
        resp = requests.post(url, headers=headers, json=body, timeout=5)
        resp.raise_for_status()
    except requests.RequestException as e:
        print(f"[SNOMED] ⚠️ Erreur HTTP pour '{term}': {e}")
        return None
        
    concepts = resp.json().get("concepts", [])

    # 2. Filtrer les concepts SNOMED CT
    snomed_concepts = [
        c for c in concepts
        if c.get("terminologyName") == "terminologie-snomed-ct-fr"
    ]
    if not snomed_concepts:
        return None

    best_coding = verify_coding(term, snomed_concepts)
    if best_coding:
        return best_coding
        
    return None

### Étape 1 – Focus sur la colonne `Allergy`

Dans un premier temps, l’analyse se concentre sur la colonne `Allergy` des données médicales. L’objectif est d’interroger le modèle Mistral afin d’extraire automatiquement des informations cliniques structurées (type, gravité, substance, manifestation…) à partir du texte libre, en vue de les transformer en ressources FHIR de type `AllergyIntolerance`.

In [11]:
# reagrder ici pour changer afin d'obtenir les fichiers du répertoire (Y:) directement sans passer par un fichier excel 

import json
import numpy as np
df = pd.read_excel(r"C:\Users\dema58815\Desktop\QualiFHIR_tests_ATCD.xlsx")


### Définition des modèles Pydantic pour structurer les résultats du LLM

Afin de structurer les réponses renvoyées par le modèle Mistral, nous définissons ici des classes Python à l’aide de la librairie `pydantic`. Ces modèles facilitent la validation, la sérialisation et la transformation des données extraites en objets exploitables dans notre pipeline.

- **`Reaction`** : représente une réaction allergique, avec la **substance** déclenchante et la **manifestation** (ou les symptômes) associés.
- **`AllergyItem`** : regroupe les caractéristiques principales d’un élément allergique :
  - `type` : `"allergy"` ou `"Pseudoallergic disposition"`ou `"Intolerance to substance"`ou `"Hypersensitivity disposition"`
  - `criticality` : gravité perçue (`low`, `high`, `unable-to-assess`)
  - `category` : type d’allergène (`medication`, `food`, `environment`, etc.)
  - `reactions` : liste des réactions observées
- **`AllergyExtraction`** : encapsule l’ensemble des éléments extraits sous forme d’une liste de `AllergyItem`.

Ces modèles servent d'interface entre le résultat textuel du LLM et la génération des ressources FHIR `AllergyIntolerance`.

In [96]:
class Reaction(BaseModel):
    substance: str
    manifestation: List[str]
    
class AllergyItem(BaseModel):
    type: str  # "Allergy" or "Pseudoallergic disposition" or "Intolerance to substance" or "Hypersensitivity disposition"
    criticality: str | None = None  # low | high | unable-to-assess
    category: str  # "medication", "food", etc.
    reactions: List[Reaction]

class AllergyExtraction(BaseModel):
    items: List[AllergyItem]

### Création de la chaîne d’extraction d’allergies à l’aide du LLM

Cette fonction `create_allergy_chain` met en place une chaîne de traitement LangChain pour interroger un modèle de langage (LLM) sur des données textuelles issues de dossiers médicaux gériatriques, dans le but d’en extraire des informations structurées sur les allergies.

#### Fonctionnement détaillé :

- **`PydanticOutputParser`** : utilisé pour imposer un format de sortie structuré basé sur le modèle `AllergyExtraction` défini précédemment. Cela permet de transformer directement la réponse du LLM en objets Python typés.
  
- **`PromptTemplate`** : le prompt est soigneusement rédigé pour guider le modèle. Il :
  - introduit le rôle du LLM comme assistant médical en gériatrie,
  - fournit un encadré contenant le texte médical à analyser (`{texte}`),
  - précise les règles de structuration des réponses (`{format_instructions}`),
  - impose de ne pas déduire ni halluciner d'informations non présentes dans le texte original.

- **Séparateurs** : des instructions explicites sont données pour reconnaître des substances séparées par des symboles comme `-`, `@`, `/` ou `,`.

- **Format attendu** : chaque allergie est décrite par :
  - `type` (allergy ou intolerance ou Intolerance to substance ou Hypersensitivity disposition),
  - `category` (e.g. medication, food…),
  - `criticality`,
  - et une ou plusieurs réactions (substance + manifestations).

- **Chaîne LangChain (`|`)** : une chaîne de transformations est construite :
  1. Le texte est injecté dans le prompt avec les consignes de formatage.
  2. Le prompt est envoyé au modèle `llm`.
  3. La sortie du modèle est analysée par le parseur Pydantic.

Le résultat final est une structure de données bien formée, directement exploitable pour la création d’une ressource FHIR `AllergyIntolerance`.

In [97]:
def create_allergy_chain(llm):
    parser = PydanticOutputParser(pydantic_object=AllergyExtraction)

    prompt = PromptTemplate.from_template( #À partir de cette information, retourne une analyse structurée au format JSON :
        """
        Tu es un.e gériatre aguéri.e. 
        Voici une information extraite du dossier patient listant les allergies du patient:
        --- 
        {texte}
        --- 
        Analyse chaque groupe de substances ( medication, food, environment, biologic) séparément, et construis une structure comme suit :

        {format_instructions}
        Certains caractères comme "-", "@", "/", ou "," doivent être compris comme des séparateurs de substances distinctes.  

        Pour chaque type de réaction, définis :
        - "type" : "Allergy" ou "Pseudoallergic disposition" ou "Intolerance to substance" ou "Hypersensitivity disposition"
        - "category" : medication, food, environment, biologic
        - "criticality" : low, high, unable-to-assess
        - "reactions" : liste d'objets avec substance, manifestation(s)

        Si aucune allergie n'est mentionnée, retourne :
        {{
            "type": "none",
            "category": "none",
            "reactions": []
        }}
        Si plusieurs substances causent la même réaction, liste-les individuellement.
        Ne complète pas avec des exemples inventés. Ne suppose pas. Ne déduis rien.
        N’inclus AUCUNE substance qui ne soit explicitement mentionnée dans le texte du dossier patient.
        """
    )

    chain = (
        {"texte": lambda x: x, "format_instructions": lambda _: parser.get_format_instructions()}
        | prompt
        | llm
        | parser
    )

    return chain

### Sélection et analyse d’un exemple de texte d’allergies

Nous sélectionnons une valeur pertinente dans la colonne `Allergies` du DataFrame. 

- Les valeurs vides ou non informatives (comme *"aucune"*, *"connue"*, *"refus"*, ou `/`) sont filtrées.
- Un exemple de texte significatif est ensuite choisi (`texte_test`) pour être analysé.

Ce texte est ensuite transmis à la chaîne `allergy_chain` précédemment construite, qui interroge le modèle LLM et renvoie un résultat structuré. Le tout est validé via le modèle Pydantic `AllergyExtraction`, garantissant la conformité du format obtenu.

In [14]:
# Récupérer une valeur non vide et pertinente
valeurs = df["Allergies"].dropna().unique()
valeurs_utiles = [v for v in valeurs if "aucune" not in v.lower() 
                                  and "connue" not in v.lower() 
                                  and "refus" not in v.lower()
                                  and v != "/"]
for i in range(len(valeurs_utiles)):
    print(i, valeurs_utiles[i])

0 Banane et kiwi
1 /!\ Réaction aux produits de contraste iodés /!\
2 - Allergie vraie sur Aspirine avec commémoratifs d'œdème de Quincke ayant nécessité une hospitalisation. - Rhabdomyolyse sévère compliquée d'une insuffisance rénale aigue terminale sur prise de Crestor.
3 Allergie à l'iode (elle aurait eu des problèmes respiratoires).
4 Pénicilline
5 -
6 Notion d'allergie à l'iode (CT abdominal en 2018)
7 Intolérance au combivent et duovent
8 Duracef, staphycid mais augmentin ok (Démangeaisons et prurit)
9 - Pénicillines, Tolérance des céphalosporines. Éviction des pénicilline (Augmentin) - Iode, Autorisation usage produit contraste iodé moyennant une prémédication par cortisone et antihistaminiques
10 sparadraps
11 allergie :  voltaren  anti-inflammatoire per os ?   reaction a l'huile essentiel de lavande 
12 Pas d'allergie documentée
13 Aïl 
14 Non documentée
15 * Iode. * Cellule micropcristaline. * Latex.
16 Notion d'allergie à la Pénicilline (pas de réaction sous Amoxiclav) et au

In [22]:
texte_test = valeurs_utiles[2]  
                                
print("Texte analysé :", texte_test)

# Charger la fonction de chaînage
allergy_chain = create_allergy_chain(model)

# Appel du LLM
result = allergy_chain.invoke(texte_test)

# Résultat validé par Pydantic
print(result)

Texte analysé : - Allergie vraie sur Aspirine avec commémoratifs d'œdème de Quincke ayant nécessité une hospitalisation. - Rhabdomyolyse sévère compliquée d'une insuffisance rénale aigue terminale sur prise de Crestor.
items=[AllergyItem(type='Allergy', criticality='high', category='medication', reactions=[Reaction(substance='Aspirine', manifestation=['œdème de Quincke', 'hospitalisation'])]), AllergyItem(type='Intolerance to substance', criticality='high', category='medication', reactions=[Reaction(substance='Crestor', manifestation=['rhabdomyolyse sévère', 'insuffisance rénale aiguë terminale'])])]


### Conversion vers des ressources FHIR `AllergyIntolerance`

Cette fonction transforme la sortie structurée du LLM (`AllergyExtraction`) en une liste de ressources FHIR prêtes à être envoyées au serveur :

- **Boucle sur chaque allergie et réaction** pour générer une ressource par substance.
- Les champs `patient`, `recorder`, `type`, `category`, `criticality`....  sont renseignés directement.
- `recordedDate` est automatiquement daté avec `datetime.now().isoformat()`.
- Si un `manifestation` est présent, il est encapsulé dans le tableau `reaction`.
- L’élément `encounter` est ajouté uniquement si un identifiant de rencontre est fourni.

Le résultat est une liste Python de dictionnaires, chacun représentant une ressource `AllergyIntolerance` valide au format FHIR.

In [23]:

"""
  609328004	http://snomed.info/sct	Allergy	
  609396006	http://snomed.info/sct	Pseudoallergic disposition	
  782197009	http://snomed.info/sct	Intolerance to substance	
  609433001	http://snomed.info/sct	Hypersensitivity disposition	
"""
def generate_allergy_resources(
    nlp_output: AllergyExtraction,
    patient_id: str,
    recorder_id: str,
    encounter_id: str | None = None
) -> List[Dict]:
    """Transforme un AllergyExtraction en une liste de ressources FHIR AllergyIntolerance."""

    resources = []


    # mapping libellé → coding SNOMED CT
    ALLERGY_TYPE_MAP = {
        "Allergy": {
            "system": "http://snomed.info/sct",
            "code": "609328004",
            "display": "Allergy"
        },
        "Pseudoallergic disposition": {
            "system": "http://snomed.info/sct",
            "code": "609396006",
            "display": "Pseudoallergic disposition"
        },
        "Intolerance to substance": {
            "system": "http://snomed.info/sct",
            "code": "782197009",
            "display": "Intolerance to substance"
        },
        "Hypersensitivity disposition": {
            "system": "http://snomed.info/sct",
            "code": "609433001",
            "display": "Hypersensitivity disposition"
        },
    }
    
    for item in nlp_output.items:
        for reaction in item.reactions:
            # Appel au serveur SMT pour chaque substance
            coding = get_snomed_coding(reaction.substance, token)
            resource = {
                "resourceType": "AllergyIntolerance",
                "meta" : {
                "profile" : [
                        "https://www.ehealth.fgov.be/standards/fhir/allergy/StructureDefinition/be-allergyintolerance"
                    ]
                  },
                #"type": item.type,
                "category": [item.category],
                "criticality": item.criticality,
                "patient": {
                    "identifier": {
                      "system": "https://www.ehealth.fgov.be/standards/fhir/core/NamingSystem/ssin",
                      "value": patient_id
                      }
                },
                "recorder": {
                    "identifier": {
                      "system": "https://www.ghdc.be/standards/fhir/NamingSystem/inami",
                      "value": recorder_id
                      }
                },
                "recordedDate": datetime.now(timezone.utc).isoformat(),
                "clinicalStatus": {"coding": [{ #on fixe ces valeurs par défaut car on n'a pas plus d'infos dans le dataset
                        "system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-clinical",
                        "code": "active"}]},
                  "verificationStatus": {"coding": [{
                        "system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-verification",
                        "code": "unconfirmed"}]},
                #"code": {
                #    "text": reaction.substance
                #}
                
            }

            # Extension be-ext-allergy-type
            chosen = item.type  
            type_coding = ALLERGY_TYPE_MAP.get(chosen)
            if type_coding:
                resource.setdefault("extension", []).append({
                    "url": "https://www.ehealth.fgov.be/StructureDefinition/be-ext-allergy-type",
                    "valueCodeableConcept": {
                        "coding": [type_coding],
                        "text": item.type
                    }
                })
            else:
                # au cas où LLM renvoie autre chose
                print(f"⚠️ Type inattendu pour be-ext-allergy-type : {chosen}")

            
            # Substance codée (si trouvée)
            if coding:
                resource["code"] = {"coding": [coding], "text": reaction.substance}
            else:
                resource["code"] = {"text": reaction.substance}

            if encounter_id:
                resource["encounter"] = {"reference": f"Encounter/{encounter_id}"}

            #  Construction de la partie reaction[]
            if reaction.manifestation:
                rxn = {
                    "substance": resource["code"],  # on reprend le même CodeableConcept que ci-dessus
                    "manifestation": []
                }
                for m in reaction.manifestation:
                    # recherche SNOMED CT pour chaque manifestation
                    mani_coding = get_snomed_coding(m, token)
                    if mani_coding:
                        rxn["manifestation"].append({
                            "coding": [mani_coding],
                            "text": m
                        })
                    else:
                        rxn["manifestation"].append({"text": m})
                resource["reaction"] = [rxn]

            resources.append(resource)


    return resources

In [ ]:
# ************************

In [ ]:
# ********************************************

In [ ]:
# la ressource va vers le repertoire  de sortie allergies 

In [28]:
import os
import json

# génération FHIR
fhir_allergies = generate_allergy_resources(
    nlp_output=result,
    patient_id="48050515361", # qualifhir test sarah
    recorder_id="15881571598",
    encounter_id="ENC123", 
    #token = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ2ZGZJdTlIZk1Ydm9ZcTFfX0UzWVI4dkFFYnI0OERXbjk5Q2R4TTNUS1BzIn0.eyJleHAiOjE3NTQzMTcyMDcsImlhdCI6MTc1NDMxMTIwNywianRpIjoiMDFlOTMyZGItMTNhMS00ZjQyLTk4ZjItNGM0NjhjOWE2MmM1IiwiaXNzIjoiaHR0cHM6Ly9zbXQuZXNhbnRlLmdvdXYuZnIvYW5zL3Nzby9hdXRoL3JlYWxtcy9BTlMiLCJhdWQiOlsib2F1dGgyLXJlc291cmNlIiwib250b3NlcnZlciIsInJlYWxtLW1hbmFnZW1lbnQiLCJhY2NvdW50Il0sInN1YiI6IjEzNjE1YTQwLWRmZGYtNDY5OC1iNDZlLTE1NjVjNjc2NTg2NyIsInR5cCI6IkJlYXJlciIsImF6cCI6InVzZXItYXBpIiwic2Vzc2lvbl9zdGF0ZSI6IjViMTUzNDgzLTMzOTUtNGYzNC1hOTZhLTdiZGFmOWE3NmRhYiIsImFsbG93ZWQtb3JpZ2lucyI6WyIvKiJdLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZGVmYXVsdC1yb2xlcy1hbnMiLCJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsib250b3NlcnZlciI6eyJyb2xlcyI6WyJncm91cGluZy9wcm90ZWN0ZWQucmVhZCIsInN5c3RlbS8qLnJlYWQiXX0sInJlYWxtLW1hbmFnZW1lbnQiOnsicm9sZXMiOlsibWFuYWdlLXVzZXJzIl19LCJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJkZWxldGUtYWNjb3VudCIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoib250b3NlcnZlci1vYXV0aC1yZXNvdXJjZS1hdWRpZW5jZSBwcm9maWxlIG9wZW5pZCBlbWFpbCBncm91cGluZy9wcm90ZWN0ZWQucmVhZCBzeXN0ZW0vKi5yZWFkIiwic2lkIjoiNWIxNTM0ODMtMzM5NS00ZjM0LWE5NmEtN2JkYWY5YTc2ZGFiIiwiY2d1IjoiMTcyMTM3NjY0NCIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiTWFyaWUgRGV0cmFpdCIsInNub21lZEFmZmlsaWF0ZU51bWJlciI6IlNOT01FRF9DVF9GUl8yMi0wNC0yMDI1XzQwZGY5ODZlNTg2NyIsInByZWZlcnJlZF91c2VybmFtZSI6Im1hcmllLmRldHJhaXRAZ2hkYy5iZSIsInNub21lZFVzZUNhc2UiOiJBY2FkZW1pcXVlOlJlY2hlcmNoZSIsImdpdmVuX25hbWUiOiJNYXJpZSIsImZhbWlseV9uYW1lIjoiRGV0cmFpdCIsImxpY2VuY2VzIjpbeyJ1cmkiOiJodHRwOi8vZGF0YS5lc2FudGUuZ291di5mci92b2NhYnVsYXJ5L2xpY2VuY2UvbGljZW5jZV9hZmZpbGlhdGlvbl9TTk9NRURfQ1QiLCJhY2NlcHRhbmNlRGF0ZSI6MTc1MTAzMjgyNDAwMH0seyJ1cmkiOiJodHRwOi8vZGF0YS5lc2FudGUuZ291di5mci92b2NhYnVsYXJ5L2xpY2VuY2UvbGljZW5jZV9uYXRpb25hbGVfU05PTUVEX0NUIiwiYWNjZXB0YW5jZURhdGUiOjE3NTEwMzI4MjQwMDB9XSwiZW1haWwiOiJtYXJpZS5kZXRyYWl0QGdoZGMuYmUiLCJzb2NpZXRlIjoiR3JhbmQgSMO0cGl0YWwgZGUgQ2hhcmxlcm9pIn0.nNW5Ld7-S4uQlCt6fwaN6jjjza_gfF0HwCWge9vTnMPM7PquAr0WEVWJTW6GruzPE9Bx1xGETOvMsGLpALZGZIdpI05TziigbMlEp_4cgvRdmZKLJx9kRc6kkz9xmfHC4uRYPqvijVsGNn7hiqL9yCE59ayZfl6OHNydFjbiT3qW55lFC8MrzDKsEUMIATsC46Dd84BUyuCIpmKJkykq6z_shYduNh5WD_9pWBw_SC8swAKaUWvRp2SoqoB8IpQ-2CyGV0iam5mxdiF91W31Li3Zh51643Tsn0rf8gUyDOjUVFcYioetzxLC8_4rSu53KjxxQ9pPo8ntkQW3wxOA_A"
)


desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
allergies_path = os.path.join(desktop_path, "allergies")
#os.makedirs(allergies_path, exist_ok=True)
output_file = os.path.join(allergies_path, "allergies_fhir_quatre.json")

# Écriture du fichier JSON
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(fhir_allergies, f, indent=2, ensure_ascii=False)

print(json.dumps(fhir_allergies))

[SNOMED-VERIFY] ATTENTION : Aucun match strict pour 'Aspirine'. Sélection du premier du top 3 : acide acétylsalicylique
[SNOMED-VERIFY] ATTENTION : Aucun match strict pour 'œdème de Quincke'. Sélection du premier du top 3 : angioœdème de Quincke
[{"resourceType": "AllergyIntolerance", "meta": {"profile": ["https://www.ehealth.fgov.be/standards/fhir/allergy/StructureDefinition/be-allergyintolerance"]}, "category": ["medication"], "criticality": "high", "patient": {"identifier": {"system": "https://www.ehealth.fgov.be/standards/fhir/core/NamingSystem/ssin", "value": "48050515361"}}, "recorder": {"identifier": {"system": "https://www.ghdc.be/standards/fhir/NamingSystem/inami", "value": "15881571598"}}, "recordedDate": "2025-09-12T08:59:30.029402+00:00", "clinicalStatus": {"coding": [{"system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-clinical", "code": "active"}]}, "verificationStatus": {"coding": [{"system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-verif

In [ ]:
# Validation avec le serveur du Ministère 
# décrire ici ce qu'on fait : 
# entrée = les fichiers allergies 
# sortie = les fichiers avec les codes validés avec le serveur termino du ministère 


In [36]:
# Config pour le bureau de l'ordi

REPERTOIRE_ENTREE = os.path.expanduser("~/Desktop/allergies")
REPERTOIRE_SORTIE = os.path.expanduser("~/Desktop/allergies_validees")
SERVER_URL = "http://10.30.3.205:8180/fhir/CodeSystem/$lookup"
EXTENSION_URL = "https://www.ghdc.be/standards/fhir/StructureDefinition/snowstorm-valide"
EXTENSION_VALUE = "validé"

# Créer le répertoire de sortie 
os.makedirs(REPERTOIRE_SORTIE, exist_ok=True)

In [57]:
def extraire_codes_snomed(ressource):
    """Extrait TOUS les codes SNOMED CT d'une ressource AllergyIntolerance."""
    codes = set()  # Utilisation d'un set pour éviter les doublons

    # 1. Code principal
    if "code" in ressource and "coding" in ressource["code"]:
        for coding in ressource["code"]["coding"]:
            if coding.get("system") == "http://snomed.info/sct":
                codes.add((coding["code"], coding.get("display", "")))

    # 2. Extensions
    if "extension" in ressource:
        for ext in ressource["extension"]:
            if "valueCodeableConcept" in ext and "coding" in ext["valueCodeableConcept"]:
                for coding in ext["valueCodeableConcept"]["coding"]:
                    if coding.get("system") == "http://snomed.info/sct":
                        codes.add((coding["code"], coding.get("display", "")))

    # 3. Réactions (substance et manifestations)
    if "reaction" in ressource:
        for reaction in ressource["reaction"]:
            # Substance
            if "substance" in reaction and "coding" in reaction["substance"]:
                for coding in reaction["substance"]["coding"]:
                    if coding.get("system") == "http://snomed.info/sct":
                        codes.add((coding["code"], coding.get("display", "")))
            # Manifestations
            if "manifestation" in reaction:
                for manifestation in reaction["manifestation"]:
                    if "coding" in manifestation:
                        for coding in manifestation["coding"]:
                            if coding.get("system") == "http://snomed.info/sct":
                                codes.add((coding["code"], coding.get("display", "")))

    return list(codes)  # Retourne une liste de tuples (code, display)


In [58]:
def valider_code_snomed(code):
    """Valide un code SNOMED CT via le serveur terminologique."""
    params = {"code": code, "system": "http://snomed.info/sct"}
    try:
        response = requests.get(SERVER_URL, params=params)
        response.raise_for_status()
        data = response.json()
        # Vérifie si la réponse contient un champ "valid"
        for param in data.get("parameter", []):
            if param.get("name") == "valid":
                return param.get("valueBoolean", False)
        return True  # Si pas de champ "valid", on suppose que le code est valide
    except Exception as e:
        print(f"Erreur lors de la validation du code {code}: {e}")
        return False


In [59]:
def ajouter_extension_validation(ressource):
    """Ajoute l'extension snowstorm_valide à la ressource."""
    if "extension" not in ressource:
        ressource["extension"] = []
    ressource["extension"].append({
        "url": EXTENSION_URL,
        "valueString": EXTENSION_VALUE
    })
    return ressource

In [60]:
def traiter_ressources():
    for filename in os.listdir(REPERTOIRE_ENTREE):
        if filename.endswith(".json"):
            filepath = os.path.join(REPERTOIRE_ENTREE, filename)
            with open(filepath, "r", encoding="utf-8") as f:
                ressources = json.load(f)
            if isinstance(ressources, list):
                ressources_modifiees = []
                for ressource in ressources:
                    # Extraction des codes
                    codes = extraire_codes_snomed(ressource)
                    # Validation de chaque code
                    validations = {}
                    for code, display in codes:
                        est_valide = valider_code_snomed(code)
                        validations[f"{code} ({display})"] = est_valide
                    ressource["_validation_snomed"] = validations
                    # Ajout de l'extension si tous les codes sont valides
                    if all(validations.values()):
                        ressource = ajouter_extension_validation(ressource)
                    ressources_modifiees.append(ressource)
                # Sauvegarde
                output_filepath = os.path.join(REPERTOIRE_SORTIE, f"valide_{filename}")
                with open(output_filepath, "w", encoding="utf-8") as f:
                    json.dump(ressources_modifiees, f, indent=2, ensure_ascii=False)
            print(f"Fichier {filename} traité.")


In [61]:
traiter_ressources()

Fichier allergies_fhir.json traité.
Fichier allergies_fhir_bis.json traité.
Fichier allergies_fhir_quatre.json traité.
Fichier allergies_fhir_trois.json traité.


In [62]:
import pandas as pd
from IPython.display import display

def afficher_validations_jupyter(ressources):
    """
    Affiche un tableau des codes SNOMED CT et leur statut de validation dans un Jupyter Notebook.
    Args:
        ressources: Liste de ressources FHIR (AllergyIntolerance) déjà traitées.
    """
    data = []
    for ressource in ressources:
        if "_validation_snomed" in ressource:
            for code_with_display, est_valide in ressource["_validation_snomed"].items():
                data.append({
                    "Code SNOMED CT": code_with_display.split(" (")[0],
                    "Display": code_with_display.split(" (")[1].rstrip(")"),
                    "Valide": "Oui" if est_valide else "Non",
                    "Ressource": ressource.get("id", "N/A")
                })
    if data:
        df = pd.DataFrame(data)
        display(df.style.applymap( 
            lambda x: 'background-color: lightgreen' if x == "Oui" else ('background-color: lightcoral' if x == "Non" else ''),
            subset=["Valide"]
        ))
    else:
        print("Aucune validation trouvée.")

In [63]:
import json
import os

# Chemin vers le fichier sur le bureau
fichier_exemple = "valide_allergies_fhir.json"

REPERTOIRE_SORTIE = os.path.expanduser("~/Desktop/allergies_validees")

# Chemin complet vers le fichier
chemin_fichier = os.path.join(REPERTOIRE_SORTIE, fichier_exemple)

# Charge les ressources validées
with open(chemin_fichier, "r", encoding="utf-8") as f:
    ressources_validees = json.load(f)

# Affiche les validations
afficher_validations_jupyter(ressources_validees)

C:\Users\dema58815\AppData\Local\Temp\ipykernel_1340\1451009765.py:22: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  display(df.style.applymap(


,Code SNOMED CT,Display,Valide,Ressource
0,387458008,acide acétylsalicylique,Oui,N/A
1,41291007,angioœdème de Quincke,Oui,N/A
2,609328004,Allergy,Oui,N/A
3,308540004,hospitalisation,Oui,N/A
4,782197009,Intolerance to substance,Oui,N/A
5,387523009,rosuvastatine,Oui,N/A
6,723189000,insuffisance rénale aiguë,Oui,N/A
7,89010004,rhabdomyolyse,Oui,N/A


In [ ]:
# je voudrais afficher la traduction 
# ce n'est pas implémenter sur le serveur donc retourne uniquement l'anglais actuellement
# ci-dessus c'est le texte du courrier ou bien ce que nous avons indiqué dans la ressource 

In [74]:

def obtenir_traductions_snomed(code_snomed, serveur_term_url):
    """
    Récupère les traductions d'un code SNOMED CT en anglais, français et néerlandais.
    Args:
        code_snomed (str): Code SNOMED CT (ex: "387458008"). # je passe le premier 
        serveur_term_url (str): URL du serveur terminologique (ex: "http://10.30.3.205:8180/fhir").
    Returns:
        dict: Dictionnaire avec les traductions {langue: traduction}.
    """
    url = f"{serveur_term_url}/CodeSystem/$lookup"
    params = {
        "system": "http://snomed.info/sct",
        "code": code_snomed
    }
    headers = {"Accept": "application/json"}

    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        data = response.json()

        traductions = {}
        for param in data.get("parameter", []):
            if param.get("name") == "designation":
                language = None
                value = None
                for part in param.get("part", []):
                    if part.get("name") == "language":
                        language = part.get("valueCode")
                    elif part.get("name") == "value":
                        value = part.get("valueString")
                if language and value:
                    traductions[language] = value

        # va retourner uniquement les langues demandées (en, fr, nl)
        result = {
            "en": traductions.get("en", "Non disponible"),
            "fr": traductions.get("fr", "Non disponible"),
            "nl": traductions.get("nl", "Non disponible")
        }
        return result
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la requête: {e}")
        return None

# Exemple d'utilisation
serveur_term_url = "http://10.30.3.205:8180/fhir"
code_snomed = "387458008"  # Exemple : le prmeier code pour "acide acétylsalicylique"

traductions = obtenir_traductions_snomed(code_snomed, serveur_term_url)
if traductions:
    print(f"Traductions pour le code {code_snomed}:")
    print(f"Anglais: {traductions['en']}")
    print(f"Français: {traductions['fr']}")
    print(f"Néerlandais: {traductions['nl']}")


Traductions pour le code 387458008:
Anglais: Acetylsalicylic acid
Français: Non disponible
Néerlandais: Non disponible


In [ ]:
# j'envoie la ressource Allergy sur le serveur de test HAPI 

In [ ]:
# creer le patient NISS : "48050515362" inventé 
Nom QUALIFHIR_TEST, LUDOVIC
Adresse RIUE DES HAIES, 229
BELGIQUE -6001 MARCINELLE
N° de dossier GHDC inventé 
Date de naissance 05/05/1948
Et ensuite essayer d'envoyer la ressource

In [ ]:
# CREER le patient 

In [30]:
# Authentification 

import requests
from requests.auth import HTTPBasicAuth

session = requests.Session()
session.headers.update({
    "Content-Type": "application/fhir+json",
    "Accept": "application/fhir+json"
})


# Coordonnées du serveur FHIR de test (Frédérik)
FHIR_BASE = "http://10.30.3.6:5080/fhir/"
HEADERS = {"Content-Type": "application/fhir+json"}


# Ressource Patient 
patient_resource = {
    "resourceType": "Patient",
    "meta": {
        "profile": [
            "https://www.ehealth.fgov.be/standards/fhir/core/StructureDefinition/be-patient"
        ]
    },
    "identifier": [
        {
            "system": "https://www.ehealth.fgov.be/standards/fhir/core/NamingSystem/ssin",
            "value": "48050515362"
        },
        {
            "system": "https://www.ghdc.be/standards/fhir/NamingSystem/DPI-XCARE",
            "value": "6073409"
        }
    ],
    "gender": "male",
    "birthDate": "1948-05-05",
    "name": [
        {
            "use": "official",
            "family": "Qualifhir_Test",
            "given": ["Ludovic"]
        }
    ]
}


# Envoi via POST
url = f"{FHIR_BASE}/Patient"
response = session.post(url, json=patient_resource)

# Diagnostic et affichage
print(f"\n HTTP {response.status_code}")
print(" Réponse brute :")
print(response.text)

if response.status_code == 201:
    try:
        data = response.json()
        print("\n Patient créé avec succès.")
        print("ID FHIR généré :", data.get("id"))
        print("Location header :", response.headers.get("Location"))
    except json.JSONDecodeError:
        print(" ATTENTION ❌ Réponse reçue mais pas en format JSON valide.")
else:
    print(" ATTENTIOB ❌ Échec de la requête.")


 HTTP 201
 Réponse brute :
{
  "resourceType": "Patient",
  "id": "335",
  "meta": {
    "versionId": "1",
    "lastUpdated": "2025-09-12T09:04:59.710+00:00",
    "source": "#idrGOrWAcRj57JiO",
    "profile": [ "https://www.ehealth.fgov.be/standards/fhir/core/StructureDefinition/be-patient" ]
  },
  "identifier": [ {
    "system": "https://www.ehealth.fgov.be/standards/fhir/core/NamingSystem/ssin",
    "value": "48050515362"
  }, {
    "system": "https://www.ghdc.be/standards/fhir/NamingSystem/DPI-XCARE",
    "value": "6073409"
  } ],
  "name": [ {
    "use": "official",
    "family": "Qualifhir_Test",
    "given": [ "Ludovic" ]
  } ],
  "gender": "male",
  "birthDate": "1948-05-05"
}

 Patient créé avec succès.
ID FHIR généré : 335
Location header : http://10.30.3.6:5080/fhir/Patient/335/_history/1


In [ ]:
# la ressource Allergyintolerance 

In [32]:
# Authentification 

import requests
from requests.auth import HTTPBasicAuth

session = requests.Session()
#session.auth = HTTPBasicAuth("marie", "AHCeDABg7WBN0MNPiSYX7")
session.headers.update({
    "Content-Type": "application/fhir+json",
    "Accept": "application/fhir+json"
})

# Coordonnées du serveur FHIR de test (Frédérik)
FHIR_BASE = "http://10.30.3.6:5080/fhir/"
HEADERS = {"Content-Type": "application/fhir+json"}


# Ressource AllergyIntolerance avec référence directe au patient
allergy_resource_1 = {
    "resourceType": "AllergyIntolerance",
    "meta": {
        "profile": [
            "https://www.ehealth.fgov.be/standards/fhir/allergy/StructureDefinition/be-allergyintolerance"
        ]
    },
    "category": ["medication"],
    "criticality": "high",
    "patient": {
        "reference": "Patient/335"
    },
    "recorder": {
        "identifier": {
            "system": "https://www.ghdc.be/standards/fhir/NamingSystem/inami",
            "value": "15881571598"
        }
    },
    "recordedDate": "2025-09-11T15:00:00+00:00",
    "clinicalStatus": {
        "coding": [
            {
                "system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-clinical",
                "code": "active"
            }
        ]
    },
    "verificationStatus": {
        "coding": [
            {
                "system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-verification",
                "code": "confirmed"
            }
        ]
    },
    "extension": [
        {
            "url": "https://www.ehealth.fgov.be/StructureDefinition/be-ext-allergy-type",
            "valueCodeableConcept": {
                "coding": [
                    {
                        "system": "http://snomed.info/sct",
                        "code": "609328004",
                        "display": "Allergy"
                    }
                ],
                "text": "Allergy"
            }
        }
    ],
    "code": {
        "coding": [
            {
                "system": "http://snomed.info/sct",
                "code": "387458008",
                "display": "acide acétylsalicylique"
            }
        ],
        "text": "Aspirine"
    },
    "reaction": [
        {
            "substance": {
                "coding": [
                    {
                        "system": "http://snomed.info/sct",
                        "code": "387458008",
                        "display": "acide acétylsalicylique"
                    }
                ],
                "text": "Aspirine"
            },
            "manifestation": [
                {
                    "coding": [
                        {
                            "system": "http://snomed.info/sct",
                            "code": "41291007",
                            "display": "angioœdème de Quincke"
                        }
                    ],
                    "text": "œdème de Quincke"
                },
                {
                    "coding": [
                        {
                            "system": "http://snomed.info/sct",
                            "code": "308540004",
                            "display": "hospitalisation"
                        }
                    ],
                    "text": "hospitalisation"
                }
            ]
        }
    ]
}

# Envoi de la ressource AllergyIntolerance
url = f"{FHIR_BASE}/AllergyIntolerance"
response = session.post(url, json=allergy_resource_1)

# Diagnostic et affichage
print(f"\nStatut HTTP : {response.status_code}")
print("Réponse brute :")
print(response.text)

if response.status_code == 201:
    try:
        data = response.json()
        print("\nAllergie créée avec succès.")
        print("ID FHIR généré :", data.get("id"))
        print("Location header :", response.headers.get("Location"))
    except json.JSONDecodeError:
        print("ATTENTION ❌ Réponse reçue mais pas en format JSON valide.")
else:
    print(f"ATTENTION ❌ Échec de la requête. Statut : {response.status_code}")
    print("Vérifiez la réponse brute pour plus de détails.")



Statut HTTP : 201
Réponse brute :
{
  "resourceType": "AllergyIntolerance",
  "id": "336",
  "meta": {
    "versionId": "1",
    "lastUpdated": "2025-09-12T09:07:04.310+00:00",
    "source": "#ZxOFUAe81fJQw1xg",
    "profile": [ "https://www.ehealth.fgov.be/standards/fhir/allergy/StructureDefinition/be-allergyintolerance" ]
  },
  "extension": [ {
    "url": "https://www.ehealth.fgov.be/StructureDefinition/be-ext-allergy-type",
    "valueCodeableConcept": {
      "coding": [ {
        "system": "http://snomed.info/sct",
        "code": "609328004",
        "display": "Allergy"
      } ],
      "text": "Allergy"
    }
  } ],
  "clinicalStatus": {
    "coding": [ {
      "system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-clinical",
      "code": "active"
    } ]
  },
  "verificationStatus": {
    "coding": [ {
      "system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-verification",
      "code": "confirmed"
    } ]
  },
  "category": [ "medication" ],
  

In [ ]:
#LE RECUP2RER 

In [34]:
if response.status_code == 201:
    location = response.headers.get("Location")
    print("Patient créé avec succès.")
    print(" Location header :", location)

    if location and "/Patient/" in location:
        patient_id = location.split("/Patient/")[1].split("/")[0]
        print(" ID FHIR généré :", patient_id)

        # Récupération de la ressource avec le bon ID
        get_url = f"{FHIR_BASE}/Patient/{patient_id}"
        get_resp = session.get(get_url)

        if get_resp.status_code == 200:
            print(" Ressource récupérée avec succès")
            print(json.dumps(get_resp.json(), indent=2))
        else:
            print(f"❌ Échec récupération : {get_resp.status_code}")
    else:
        print(" Header 'Location' manquant ou mal formé ou bien sur 201.")

Patient créé avec succès.
 Location header : http://10.30.3.6:5080/fhir/AllergyIntolerance/336/_history/1
 Header 'Location' manquant ou mal formé ou bien sur 201.


In [ ]:
# et je récupère l'allergie 

In [35]:
import requests
from requests.auth import HTTPBasicAuth
import json

# Configuration de la session
session = requests.Session()
session.headers.update({
    "Accept": "application/fhir+json"
})


# ID FHIR du patient
patient_id = "335"

# URL de recherche des allergies pour ce patient
search_url = f"{FHIR_BASE}/AllergyIntolerance?patient={patient_id}"

# Envoi de la requête GET
response = session.get(search_url)

# Affichage des résultats
print(f"\nStatut HTTP : {response.status_code}")
print("Réponse brute :")
print(response.text)

if response.status_code == 200:
    try:
        bundle = response.json()
        print("\nAllergies trouvées pour le patient (ID FHIR = 335) :")
        for entry in bundle.get("entry", []):
            allergy = entry.get("resource")
            print(f"\nID de l'allergie : {allergy.get('id')}")
            print(f"Substance : {allergy.get('code', {}).get('text')}")
            print(f"Type : {allergy.get('extension', [{}])[0].get('valueCodeableConcept', {}).get('text')}")
            print(f"Réactions : {[m.get('text') for m in allergy.get('reaction', [{}])[0].get('manifestation', [])]}")
    except json.JSONDecodeError:
        print("ATTENTION ❌ Réponse reçue mais pas en format JSON valide.")
else:
    print(f"ATTENTION ❌ Échec de la requête. Statut : {response.status_code}")



Statut HTTP : 200
Réponse brute :
{
  "resourceType": "Bundle",
  "id": "01ce1176-f039-4e73-a1f2-f6e4fd3b4f8a",
  "meta": {
    "lastUpdated": "2025-09-12T09:11:06.929+00:00"
  },
  "type": "searchset",
  "total": 1,
  "link": [ {
    "relation": "self",
    "url": "http://10.30.3.6:5080/fhir/AllergyIntolerance?patient=335"
  } ],
  "entry": [ {
    "fullUrl": "http://10.30.3.6:5080/fhir/AllergyIntolerance/336",
    "resource": {
      "resourceType": "AllergyIntolerance",
      "id": "336",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2025-09-12T09:07:04.310+00:00",
        "source": "#ZxOFUAe81fJQw1xg",
        "profile": [ "https://www.ehealth.fgov.be/standards/fhir/allergy/StructureDefinition/be-allergyintolerance" ]
      },
      "extension": [ {
        "url": "https://www.ehealth.fgov.be/StructureDefinition/be-ext-allergy-type",
        "valueCodeableConcept": {
          "coding": [ {
            "system": "http://snomed.info/sct",
            "code": "60

In [ ]:
# FIN 

In [ ]:
# suite disponible si besoin voir Marie Detrait 

### Conclusion

Ce pipeline complet permet d'extraire, structurer et transformer automatiquement des données médicales gériatriques hétérogènes en ressources FHIR standardisées, prêtes à être intégrées dans un entrepôt ou un serveur de santé.

Les principales étapes couvertes sont :

- **Extraction par LLM** de concepts médicaux à partir de texte libre (allergies, pathologies, observations, traitements, procédures).
- **Structuration Pydantic** des résultats selon les modèles FHIR (`AllergyIntolerance`, `Condition`, `Observation`, `MedicationStatement`, `Procedure`, `Encounter`).
- **Codage SNOMED CT** semi-automatique des concepts cliniques via le serveur SMT de l'ANS.
- **Génération et validation de Bundles FHIR**, avec possibilité de vérification via `$validate`.
- **Transmission transactionnelle** des ressources vers un serveur FHIR avec gestion des erreurs, journalisation et suivi précis.

L’ensemble du pipeline facilite l’intégration de données textuelles gériatriques dans un cadre interopérable, standardisé et exploitable pour le suivi clinique, la recherche ou l’analyse secondaire.

In [ ]:
# FIN